In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile
import random
import shutil
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Input, GRU
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


Mounted at /content/drive


In [3]:
# Paths
DATA_ZIP_PATH = "/content/drive/MyDrive/GoatDataset.zip"
EXTRACT_DIR = "/content/drive/MyDrive/GoatDataset"

with zipfile.ZipFile(DATA_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)
print("✅ Dataset unzipped successfully")

# Original folders
ORIGINAL_HEALTHY = os.path.join(EXTRACT_DIR, "healthy_goat")
ORIGINAL_UNHEALTHY = os.path.join(EXTRACT_DIR, "unhealthy_goat")

# Split folders
BASE_DIR = "/content/drive/MyDrive/Goat_Splits"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

for folder in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    for cls in ["healthy_goat", "unhealthy_goat"]:
        os.makedirs(os.path.join(folder, cls), exist_ok=True)

# Function to split data
def split_data(source_dir, train_dir, val_dir, test_dir, split=(0.7, 0.15, 0.15)):
    files = os.listdir(source_dir)
    random.shuffle(files)
    n_total = len(files)
    n_train = int(split[0]*n_total)
    n_val = int(split[1]*n_total)

    for i, file in enumerate(files):
        src_path = os.path.join(source_dir, file)
        if i < n_train:
            dst_path = os.path.join(train_dir, file)
        elif i < n_train + n_val:
            dst_path = os.path.join(val_dir, file)
        else:
            dst_path = os.path.join(test_dir, file)
        shutil.copy(src_path, dst_path)

split_data(ORIGINAL_HEALTHY,
           os.path.join(TRAIN_DIR, "healthy_goat"),
           os.path.join(VAL_DIR, "healthy_goat"),
           os.path.join(TEST_DIR, "healthy_goat"))

split_data(ORIGINAL_UNHEALTHY,
           os.path.join(TRAIN_DIR, "unhealthy_goat"),
           os.path.join(VAL_DIR, "unhealthy_goat"),
           os.path.join(TEST_DIR, "unhealthy_goat"))

print("✅ Train/Val/Test splits created")


✅ Dataset unzipped successfully
✅ Train/Val/Test splits created


In [4]:
def check_dataset_splits(base_dir):
    print(f"Checking dataset in: {base_dir}\n")
    for split in ["train", "val", "test"]:
        split_dir = os.path.join(base_dir, split)
        print(f"--- {split.upper()} ---")
        for cls in os.listdir(split_dir):
            cls_dir = os.path.join(split_dir, cls)
            num_files = len(os.listdir(cls_dir))
            print(f"{cls}: {num_files} images")
        print()

# Run the check
check_dataset_splits(BASE_DIR)


Checking dataset in: /content/drive/MyDrive/Goat_Splits

--- TRAIN ---
healthy_goat: 312 images
unhealthy_goat: 341 images

--- VAL ---
healthy_goat: 66 images
unhealthy_goat: 73 images

--- TEST ---
healthy_goat: 68 images
unhealthy_goat: 74 images



In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Load datasets
train_ds = image_dataset_from_directory(
    TRAIN_DIR, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', seed=42
)
val_ds = image_dataset_from_directory(
    VAL_DIR, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', seed=42
)
test_ds = image_dataset_from_directory(
    TEST_DIR, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical', shuffle=False
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Detected Classes:", class_names)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomTranslation(0.1, 0.1)
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
val_ds = val_ds.map(lambda x, y: (x/255.0, y))
test_ds = test_ds.map(lambda x, y: (x/255.0, y))

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)


In [ ]:
inputs = Input(shape=(128, 128, 3))

x = Conv2D(32, 3, activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D(2)(x)

x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2)(x)

x = Conv2D(128, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2)(x)

x = Conv2D(256, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2)(x)

# Reshape for RNN
x = tf.keras.layers.Reshape((8, 256*8))(x)
x = GRU(256, return_sequences=False)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
EPOCHS = 20

early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_goat_crnn_model.keras', monitor='val_accuracy', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stop, checkpoint, reduce_lr]
)


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Goat_CRNN_Model.keras"
model.save(MODEL_PATH)
print("✅ Model saved successfully")

test_loss, test_acc = model.evaluate(test_ds)
print(f"🔥 Test Accuracy: {test_acc*100:.2f}%")

# Confusion matrix
y_true, y_pred = [], []
for x, y in test_ds:
    preds = model.predict(x)
    y_true.extend(np.argmax(y.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

cm = tf.math.confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,8))
sns.heatmap(cm, xticklabels=class_names, yticklabels=class_names, cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
IMAGE_PATH = "/content/drive/MyDrive/test_image/goat_test1.png"

# Load model
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded successfully")

# Load & preprocess image
img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=IMG_SIZE)
img_arr = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_arr = np.expand_dims(img_arr, axis=0)

# Predict
pred = model.predict(img_arr)
idx = np.argmax(pred[0])
predicted_class = class_names[idx]
confidence = np.max(pred[0]) * 100
status = "HEALTHY ✅" if "healthy" in predicted_class.lower() else "DISEASED ❌"

# Show result
plt.imshow(img)
plt.axis("off")
plt.title(f"{predicted_class}\n{status} ({confidence:.2f}%)")
plt.show()

print("Predicted Class :", predicted_class)
print("Status          :", status)
print(f"Confidence      : {confidence:.2f}%")
